In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import scipy

import google.colab
google.colab.drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
materials_hier = pd.read_csv('/content/drive/My Drive/LentaHack/materials.csv')
materials_hier.head()

,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0


In [3]:
materials_ids = pd.read_csv('/content/drive/My Drive/LentaHack/materials_ids.csv').drop(columns=['Unnamed: 0'])
materials_ids.head()

,material,material_id
0,35cbdf61de9e19c8b417327aaef14c88,21956
1,c0b0bf24d4ec71da3d304f761ec555d8,79256
2,f0fc5e654a81a7c4b8ba8d7c26546e14,99321
3,353693e64fb5f9e2d29746d7fe6edf1e,21739
4,5d9e0b4302ce95448cae72165ff4cf5b,38443


In [4]:
clients_ids = pd.read_csv('/content/drive/My Drive/LentaHack/customers_ids.csv').drop(columns=['Unnamed: 0'])
clients_ids.head()

,client_id_num,client_id
0,0,0000d48a8891cd462fdf01ea861ec53e
1,1,00028a1dcb30863404a386961a69300e
2,2,0004274683494a9eca5e70ef97299d55
3,3,00072f81efcb3481861f7300c5e2a88a
4,4,00076596eb749cf10c2e984621dffe95


In [87]:
#data = pd.read_csv('/content/drive/My Drive/LentaHack/data_train.csv').drop(columns=['Unnamed: 0']).dropna()
#data = data.rename(columns={"harmonic_mean_loyalty": "loyalty", "client_id": "client_id_str", 'client_id_num': 'client_id'})
data_csr = scipy.sparse.csr_matrix((data.loyalty, (data.client_id, data.material_id)))
data.head()

,client_id_str,hier_level_4,material,loyalty,material_id,client_id
0,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.030925,43194,0
1,0000d48a8891cd462fdf01ea861ec53e,0dea9ce37d6d69a8bbc3e1c5e8ef6c9e,27297a3dcebad83fdd736facdae40caf,0.007822,15859,0
2,0000d48a8891cd462fdf01ea861ec53e,14df794291216db63fdeb9dfc9bf350c,6dc38377b90a235304ff4ac87a8e905c,0.007822,45097,0
3,0000d48a8891cd462fdf01ea861ec53e,17bdeb073cbf020103735133de8927f8,7eccfa30cf0507106febd34aad10ef00,0.007822,52097,0
4,0000d48a8891cd462fdf01ea861ec53e,184f06b1548d70c83bd10a6b5dd8c965,ba02089940741339300b82e24e91197d,0.007822,76572,0


In [89]:
from sklearn.neighbors import NearestNeighbors


""""""""""""""""""""""""""""""""
"""""""FIT CLASSIFAER """""""""

""""""""""""""""""""""""""""""""

KNN = NearestNeighbors(n_neighbors=10)
KNN.fit(data_csr)

client_row = scipy.sparse.csr_matrix((data[data['client_id']==0].loyalty, 
                                   (data[data['client_id']==0].client_id, data[data['client_id']==0].material_id)), 
                                    shape = (1,105609))

client_row = client_row.toarray().reshape(1,-1)
neighs = KNN.kneighbors(client_row)
print(neighs)


(array([[0.        , 0.37892768, 0.38622771, 0.39185912, 0.39699531,
        0.39735918, 0.39749011, 0.39755529, 0.3982522 , 0.40159841]]), array([[    0, 36539, 12763, 27086, 26451, 12524, 46514,  9909, 38660,
        16260]]))


In [65]:
data[data['client_id'] == '0004274683494a9eca5e70ef97299d55'].client_id_num.values[0]

2

In [90]:
def find_replacement(client_id):

      # convert client_id to client_idx
      client_idx = clients_ids[clients_ids['client_id'] == client_id].client_id_num.values[0]

      # detect his key-prodcut
      client = data[data.client_id == 0]
      max_loyalty = client.loyalty.values.max()
      max_loyalty_material_idx = int(client.iloc[client.loyalty.idxmax()].material_id)

      # find his kNN
      client_row = scipy.sparse.csr_matrix((client.loyalty, 
                                        (client.client_id, client.material_id)), 
                                          shape = (1,105609))

      client_row = client_row.toarray()
      neighs = KNN.kneighbors(client_row)[1][0][1:]

      # find key-product (absent) material_id and hier_level_4
      absent_material = materials_ids[materials_ids.material_id == max_loyalty_material_idx].material.values[0]
      absent_material_h = materials_hier[materials_hier.material == absent_material].hier_level_4.values[0]

      # iterante over kNN
      for n, neigh in enumerate(neighs):
          
          # find neighbors top products in category of user key-product
          mat_ids = data[data['client_id']==neigh].sort_values(by=['loyalty'], ascending=False).material_id.values

          # search amoung neighbors top product in category of user key-product
          for i, mat_id in enumerate(mat_ids):
                  material_n = materials_ids[materials_ids.material_id == mat_id].material.values[0]
                  if materials_hier[materials_hier.material == material_n].hier_level_4.values[0] == absent_material_h:

                          if material_n != absent_material:
                            #print(material_n, n, i)
                            return material_n

2
0.1565446961832537
286


IndexError: ignored

In [91]:
# EXAMPLE
import time

start = time.time()

client_id = '0004274683494a9eca5e70ef97299d55'

rp = find_replacement(client_id)


In [28]:
#test_data = pd.read_csv('/content/drive/My Drive/LentaHack/test.csv')
clients = test_data.client_id.unique()
test_data

,Unnamed: 0,client_id,material,sales_count_x,hier_level_4,sales_count_y
0,0,0000d48a8891cd462fdf01ea861ec53e,00a84aa181787e5c6c066e4893101eff,0.486,a8c04add9dedf192fb9141bd41d4c85d,1.152
1,1,0000d48a8891cd462fdf01ea861ec53e,2f0a52aa937c0e9378d4ff6f2f6dfdb7,0.228,a8c04add9dedf192fb9141bd41d4c85d,1.152
2,2,0000d48a8891cd462fdf01ea861ec53e,af7bdccc064ef335c1cd0210a9d0f395,0.438,a8c04add9dedf192fb9141bd41d4c85d,1.152
3,3,0000d48a8891cd462fdf01ea861ec53e,00c5d51c9029b72cfe32b3935f4685ac,1.000,62d16a465495a62e67183681ae33aa99,2.000
4,4,0000d48a8891cd462fdf01ea861ec53e,5f3ccff53da9a228bebcba1eeac94010,1.000,62d16a465495a62e67183681ae33aa99,2.000
...,...,...,...,...,...,...
6035350,6035350,fffd3e78e3ea6bf00aee1372efc09dad,b1d7591d2e8ab3c709c3c53c4094f705,1.000,86c82bb5cc4a93ac4d579cdc5d75c5bc,1.000
6035351,6035351,fffd3e78e3ea6bf00aee1372efc09dad,be497986bd5afd60acc5b5b0e7bdfcee,1.000,f1471bc23e88cb9d971953f99e020188,1.000
6035352,6035352,fffd3e78e3ea6bf00aee1372efc09dad,bf868ab23a696dac26535b70ffb5b047,2.000,807f195307dc092a426eda79b4da398f,2.000
6035353,6035353,fffd3e78e3ea6bf00aee1372efc09dad,ccb92be59ce7e5f3371b0dea5e871a9b,1.000,dd1543656cc988aac30ead1562ec6e4d,1.000


In [43]:
!pip install IPython
from IPython.display import clear_output


In [ ]:
from tqdm import tqdm

count = 0
for i, client in enumerate(tqdm(clients)):
    replacement = find_replacement(client)
    count += replacement in test_data[test_data['client_id'] == client].material.unique()
    if i % 50 == 0:
      print(i, count)


In [35]:
replacement in test_data.material.unique()

True